In [2]:
# load the packages
import os
import yaml
import pandas as pd

# load the runner 
from finetune_transformer import FineTune

In [ ]:
# load config file
config = yaml.load(open("config_ft_transformer.yaml", "r"), Loader=yaml.FullLoader)
config['dataloader']['randomSeed'] = 0
print(config)

if 'hMOF' in config['dataset']['data_name']:
    task_name = config['dataset']['data_name']
    pressure = config['dataset']['data_name'].split('_')[-1]
if 'QMOF' in config['dataset']['data_name']:
    task_name = 'QMOF'

# ftf: finetuning from
# ptw: pre-trained with
if config['fine_tune_from'] == 'scratch':
    ftf = 'scratch'
    ptw = 'scratch'
else:
    ftf = config['fine_tune_from'].split('/')[-1]
    ptw = config['trained_with']

seed = config['dataloader']['randomSeed']

log_dir = os.path.join(
    'training_results/finetuning/Transformer',
    'Trans_{}_{}_{}'.format(ptw,task_name,seed)
)

if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# run the training on dataset
fine_tune = FineTune(config, log_dir)
fine_tune.train()
loss, metric = fine_tune.test()

# save the training results
fn = 'Trans_{}_{}_{}.csv'.format(ptw,task_name,seed)
print(fn)
df = pd.DataFrame([[loss, metric.item()]])
df.to_csv(
    os.path.join(log_dir, fn),
    mode='a', index=False, header=False
)

{'batch_size': 64, 'epochs': 1, 'eval_every_n_epochs': 1, 'fine_tune_from': './training_results/pretraining', 'trained_with': 'CGCNN', 'log_every_n_steps': 50, 'gpu': 'cuda:0', 'vocab_path': 'tokenizer/vocab_full.txt', 'cuda': True, 'num_workers': 0, 'task': 'regression', 'optim': {'optimizer': 'Adam', 'init_lr': 5e-05, 'weight_decay': '1e-6'}, 'dataloader': {'valid_ratio': 0.15, 'test_ratio': 0.15, 'use_ratio': 1, 'randomSeed': 0}, 'dataset': {'data_name': 'QMOF', 'dataPath': './benchmark_datasets/QMOF/mofid/QMOF_small_mofid.csv'}, 'Transformer': {'ntoken': 4021, 'd_model': 512, 'nhead': 8, 'd_hid': 512, 'nlayers': 6, 'dropout': 0.1}}
Running on: cpu
The random seed is:  0
Train size: 5226, Validation size: 1119, Test size: 1119
Pre-trained weights not found. Training from scratch.
Epoch: 1, Batch: 0, Loss: 0.9366955757141113
